In [2]:
import pandas as pd
import numpy as np


In [24]:
# load in all csv in dir.
import glob 
path = r'.' 
all_csv = glob.glob(path + '/*.csv')
# this will be the master frame that holds each CSV eventually
frame = pd.DataFrame()
frame_alt = pd.DataFrame()
list_ = []
list_alt = []
for file_ in all_csv:
    
    # fixme: can omit index change here and pivot later with GHCN ID as index.
    df = pd.read_csv(file_, header=1) #index_col='GHCN ID',
    df.head()
    
    # remove some columns
    df.drop(df.columns[22:], axis=1, inplace=True)
    df.drop(df.columns[6:18], axis=1, inplace=True)
    df.drop(df.columns[[3]], axis=1, inplace=True)
    
    # add state in as a convenience
    state = file_.split('/')[-1:]
    df.insert(1, 'State', state[0][0:2])
    
    # fix the letter values, T is trace, M is missing(?) 
    datecols = df.columns[6:10]
    df[datecols] = df[datecols].replace(['M', 'T'], 0.0)
    
    # adjust col names to dates
    year = '2017'
    new_cols = {day: day[-2:] + '/03/' + year for day in datecols}
    df.rename(columns=new_cols, inplace=True)
    
    # now add this df to main dataframe
    list_.append(df)
    
    # melt (stack the DF)
    tidy = pd.melt(df, \
                   id_vars=['GHCN ID', 'State', 'Station Name','County','Latitude','Longitude'],\
                   var_name='Date',\
                   value_name='Amount')
    list_alt.append(tidy)
    
frame = pd.concat(list_)
frame_alt = pd.concat(list_alt)
frame.head()

frame_alt.head(n=20)

,GHCN ID,State,Station Name,County,Latitude,Longitude,Date,Amount
0,USC00060227,CT,BAKERSVILLE,LITCHFIELD,-73.01,41.84,13/03/2017,0.0
1,USC00060299,CT,BARKHAMSTED,LITCHFIELD,-72.96,41.92,13/03/2017,0
2,US1CTHR0018,CT,BERLIN 2.4 SSE,HARTFORD,-72.76,41.58,13/03/2017,0.0
3,USC00060808,CT,BRIDGEPORT-SUCCESS HILL,FAIRFIELD,-73.16,41.20,13/03/2017,0.0
4,US1CTFR0009,CT,BROOKFIELD 3.3 SSE,FAIRFIELD,-73.39,41.44,13/03/2017,0.0
5,USC00060973,CT,BURLINGTON,HARTFORD,-72.93,41.79,13/03/2017,0
6,US1CTHR0007,CT,CENTRAL MANCHESTER 2.7 SW,HARTFORD,-72.55,41.75,13/03/2017,0
7,US1CTHR0024,CT,COLLINSVILLE 0.9 NW,HARTFORD,-72.94,41.83,13/03/2017,0.0
8,US1CTWN0006,CT,DAYVILLE 2.0 ENE,WINDHAM,-71.85,41.86,13/03/2017,0.0
9,US1CTHR0022,CT,EAST HARTFORD 1.3 E,HARTFORD,-72.59,41.76,13/03/2017,0.0


In [14]:
frame_alt.groupby('State').first()

,GHCN ID,Station Name,County,Latitude,Longitude,Date,Amount
State,,,,,,,
CT,USC00060227,BAKERSVILLE,LITCHFIELD,-73.01,41.84,13/03/2017,0.0
DE,US1DEKN0012,CAMDEN 3.7 ESE,KENT,-75.49,39.08,13/03/2017,0.0
MA,US1MAPL0015,ABINGTON 1.2 NNE,PLYMOUTH,-70.95,42.13,13/03/2017,0.0
MD,USW00013701,ABERDEEN PHILLIPS FLD,HARFORD,-76.17,39.47,13/03/2017,0.0
ME,US1MEPS0009,ABBOT 4.6 WNW,PISCATAQUIS,-69.55,45.22,13/03/2017,0
NH,US1NHHL0025,AMHERST 3.7 NNE,HILLSBOROUGH,-71.61,42.91,13/03/2017,0
NJ,US1NJSS0001,ANDOVER TWP 1.7 W,SUSSEX,-74.76,41.02,13/03/2017,0.0
NY,USC00300023,ADDISON,STEUBEN,-77.23,42.10,13/03/2017,0.0
PA,US1PAAD0002,ABBOTTSTOWN 2.4 N,ADAMS,-76.99,39.92,13/03/2017,0.0


In [17]:
frame_alt['State'].value_counts()
frame_alt['Station Name'].nunique()
frame_alt.count()



GHCN ID         7088
State           7088
Station Name    7088
County          7088
Latitude        7088
Longitude       7088
Date            7088
Amount          7088
dtype: int64

In [26]:
# FIXME: add a column of date and pivot the final columns on this.
import dateutil

# Convert date from string to date times
#frame['date'] = frame['date'].apply(dateutil.parser.parse, dayfirst=True)

# save and reload to check:
output = 'output/merged_alt.csv'
frame_alt.to_csv(output, index=False)
frame_alt.head()
frame_alt = pd.read_csv(output)
frame_alt.tail()

,GHCN ID,State,Station Name,County,Latitude,Longitude,Date,Amount
7083,US1VTOL0001,VT,WESTFIELD 0.7 WNW,ORLEANS,-72.44,44.89,16/03/2017,8.5
7084,US1VTWH0006,VT,WESTMINSTER WEST 0.9 E,WINDHAM,-72.52,43.07,16/03/2017,0.0
7085,US1VTCL0018,VT,WHEELOCK 1.6 S,CALEDONIA,-72.09,44.56,16/03/2017,1.6
7086,USC00439984,VT,WOODSTOCK,WINDSOR,-72.51,43.63,16/03/2017,0.2
7087,USC00439988,VT,WORCESTER 2 W,WASHINGTON,-72.58,44.37,16/03/2017,3.5
